In [42]:
%pip install scikeras

Note: you may need to restart the kernel to use updated packages.


In [43]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier  # Updated import
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle


In [37]:
# Load data file
data = pd.read_csv('Churn_Modelling.csv')

# Preprocess the data
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.head()

# Encode Categorical variables
label_endoder_gender = LabelEncoder()
data['Gender']= label_endoder_gender.fit_transform(data['Gender'])

# One-hot Encode 'Geography'
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder(handle_unknown = 'ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

# Combine one-hot encoded columns with original data
data=pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

# Split the data into features and target

X = data.drop('Exited', axis=1)
y = data['Exited']

# Split the data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

# Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save the encoders and Scalers in Pickle file, because we have to use it later
with open('label_endoder_gender.pkl', 'wb')as file:
    pickle.dump(label_endoder_gender,file)
    
with open('onehot_encoder_geo.pkl', 'wb')as file:
    pickle.dump(onehot_encoder_geo,file)
    
with open('scaler.pkl', 'wb')as file:
    pickle.dump(scaler,file)
    
    

In [38]:
# Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32, layers=1):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))
    
    # Add hidden layers (note: fixed your hidden layer neurons to match first layer)
    for _ in range(layers-1):
        model.add(Dense(neurons, activation='relu'))  # Changed from 1 to neurons
    
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model


In [39]:
# 2. Create KerasClassifier with model (not build_fn)
model = KerasClassifier(
    model=create_model,  # Note: 'model' instead of 'build_fn'
    neurons=32,         # Default values must match create_model defaults
    layers=1,           # Default values must match create_model defaults
    optimizer='adam',   # Added optimizer here
    epochs=50,
    batch_size=10,
    verbose=0
)


In [40]:
# 3. Define param_grid (now properly mapped)
param_grid = {
    'model__neurons': [16, 32, 64, 128],  # Note the model__ prefix
    'model__layers': [1, 2],               # Note the model__ prefix
    'epochs': [50, 100]
}


In [ ]:
# 4. Perform Grid Search
grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    n_jobs=-1,
    cv=3,
    verbose=1
    verbose=1
    verbose=1
)
grid_result = grid.fit(X_train, y_train)

# Print results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


m:\ML_Practice\ML_Project_First\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Best: 0.858000 using {'epochs': 50, 'model__layers': 1, 'model__neurons': 16}
